In [174]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne
import pywt

In [3]:
def file_to_DataDrame(path):
    """
    This function takes in a file path and returns a dataframe with the data and the target values
    format:
        Fc5	        Fc3	        Fc1	        ...	Oz	        O2	        Iz	        target
    0	-0.000046	-0.000041	-0.000032	...	0.000040	0.000108	0.000055	0
    1	-0.000054	-0.000048	-0.000034	...	0.000064	0.000114	0.000074	0
    ...
    """

    reader = mne.io.read_raw_edf(path, preload=True)
    annotations = reader.annotations  # get the values of the annotations
    codes = annotations.description  # get the codes from the annotations

    df = pd.DataFrame(
        reader.get_data().T,
        columns=[channel.replace(".", "") for channel in reader.ch_names],
    )  # transpose the data to get the right shape
    df = df[~(df == 0).all(axis=1)]  # remove rows with all zeros
    timeArray = np.array(
        [round(x, 10) for x in np.arange(0, len(df) / 160, 0.00625)]
    )  # create an array of time values

    codeArray = []
    counter = 0
    for timeVal in timeArray:
        if (
            timeVal in annotations.onset
        ):  # if the time value is in the onset array, add the corresponding code to the codeArray
            counter += 1
        code_of_target = int(
            codes[counter - 1].replace("T", "")
        )  # convert T0 to 0, T1 to 1, etc
        codeArray.append(code_of_target)

    df["target"] = np.array(codeArray).T
    return df

In [142]:
import os

In [149]:
def read_all_file_df(num_exp=[3, 4], num_people=10):
    """    condct all files in one dataframe"""
    all_df = pd.DataFrame()
    for subject in range(1, num_people):
        for file in num_exp:
            fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"
            df = file_to_DataDrame(fileName)
            all_df = pd.concat([all_df, df], axis=0)
    return all_df


In [156]:

df = read_all_file_df([3,7,11],109)


Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S001/S001R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S001/S001R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S002/S002R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracti

/tmp/ipykernel_547721/1481808001.py:11: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)


Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


/tmp/ipykernel_547721/1481808001.py:11: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)


Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


/tmp/ipykernel_547721/1481808001.py:11: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)


Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S101/S101R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S101/S101R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S101/S101R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S102/S102R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracti

In [151]:
class EEGDataset(torch.utils.data.Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

In [160]:
eeg_ds=EEGDataset(df.iloc[:, :-1].values, df.iloc[:, -1].values)

In [164]:
import pickle

def save_to_pickle(data, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

In [165]:
save_to_pickle(eeg_ds, './pkl/eeg_ds.pkl')

In [ ]:
load_from_pickle('./pkl/eeg_ds.pkl')

In [167]:
signal = df.iloc[:4000,:-1].values

In [186]:
time= np.linspace(0, len(signal)/160, len(signal))
widths = np.geomspace(1, 1024, num=10)
sampling_period = sampling_period = np.diff(time).mean()
cwtmatr, freqs =pywt.cwt(signal, widths,'cgau4', sampling_period=sampling_period)
cwtmatr= torch.tensor(cwtmatr)


In [187]:
cwtmatr.shape

torch.Size([100, 4000, 64])

In [189]:
cwtmatr= cwtmatr.permute(1,2,0)

In [190]:
cwtmatr.shape

torch.Size([4000, 64, 100])

In [184]:
#cwtmatr to df 
cwt_df = pd.DataFrame(cwtmatr[0].numpy())

array([7.99800000e+01, 7.45717491e+01, 6.95292043e+01, 6.48276366e+01,
       6.04439891e+01, 5.63567641e+01, 5.25459173e+01, 4.89927602e+01,
       4.56798677e+01, 4.25909932e+01, 3.97109884e+01, 3.70257297e+01,
       3.45220484e+01, 3.21876660e+01, 3.00111347e+01, 2.79817805e+01,
       2.60896514e+01, 2.43254680e+01, 2.26805789e+01, 2.11469172e+01,
       1.97169618e+01, 1.83837000e+01, 1.71405934e+01, 1.59815458e+01,
       1.49008730e+01, 1.38932754e+01, 1.29538115e+01, 1.20778742e+01,
       1.12611679e+01, 1.04996872e+01, 9.78969794e+00, 9.12771815e+00,
       8.51050147e+00, 7.93502101e+00, 7.39845456e+00, 6.89817075e+00,
       6.43171616e+00, 5.99680324e+00, 5.59129916e+00, 5.21321528e+00,
       4.86069744e+00, 4.53201688e+00, 4.22556171e+00, 3.93982905e+00,
       3.67341765e+00, 3.42502099e+00, 3.19342093e+00, 2.97748167e+00,
       2.77614424e+00, 2.58842125e+00, 2.41339210e+00, 2.25019842e+00,
       2.09803991e+00, 1.95617037e+00, 1.82389404e+00, 1.70056225e+00,
      

In [181]:
cwt_fq = torch.empty(100,1)

In [193]:
type(cwtmatr[0][0][0]) 



torch.Tensor

In [194]:
class CWTDataset(torch.utils.data.Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target
    
    def transform(self, signal):
        time= np.linspace(0, len(signal)/160, len(signal))
        widths = np.geomspace(1, 1024, num=100)
        sampling_period = sampling_period = np.diff(time).mean()
        cwtmatr, _ = pywt.cwt(signal, widths,'cgau4', sampling_period=sampling_period)
        return cwtmatr

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.transform(self.data[idx-640:idx])
        return data, self.target[idx]

In [178]:
cwt_df = pd.DataFrame(cwtmatr.T, columns=[f'cwt_{i}' for i in range(cwtmatr.shape[0])])

ValueError: Must pass 2-d input. shape=(64, 4000, 100)

In [ ]:

time = np.linspace(0, 4000 / 160, 4000)  # time of the signal
signal = fc5
widths = np.geomspace(1, 1024, num=100)  # range of scales
sampling_period = np.diff(time).mean()  # 0.006251562890722681
print(sampling_period)
wavlist = pywt.wavelist(kind="continuous")
for wavelet in wavlist:
    # A few wavelet families require parameters in the string name
    if wavelet in ["cmor", "shan"]:
        wavelet += "1-1"
    elif wavelet == "fbsp":
        wavelet += "1-1.5-1.0"

    # compute the wavelet transform
    cwtmatr, freqs = pywt.cwt(signal, widths, wavelet, sampling_period=sampling_period)
    # cwtmatr= np.abs(cwtmatr[:-1,:-1])
    cwtmatr = np.abs(cwtmatr)
    # plot the wavelet transform
    plt.figure(figsize=(20, 3))
    plt.pcolormesh(time, freqs, cwtmatr)

    maxval = np.max(freqs)
    plt.vlines(x=xT0, ymin=0, ymax=maxval, color="r", label="T0", linestyles="dashed")
    plt.vlines(x=xT1, ymin=0, ymax=maxval, color="c", label="T1", linestyles="dashed")
    plt.vlines(x=xT2, ymin=0, ymax=maxval, color="y", label="T2", linestyles="dashed")
    plt.yscale("log")
    plt.ylabel("Frequency [Hz]")
    plt.xlabel("Time [s]")
    plt.title(f"Wavelet Transform with {wavelet}")
    plt.colorbar()
    plt.show()

In [13]:


def file_to_tensor(path):


    reader = mne.io.read_raw_edf(path, preload=True)
    annotations = reader.annotations  # get the values of the annotations
    codes = annotations.description  # get the codes from the annotations

    df = pd.DataFrame(
        reader.get_data().T,
        columns=[channel.replace(".", "") for channel in reader.ch_names],
    )  # transpose the data to get the right shape
    df = df[~(df == 0).all(axis=1)]  # remove rows with all zeros
    timeArray = np.array(
        [round(x, 10) for x in np.arange(0, len(df) / 160, 0.00625)]
    )  # create an array of time values

    codeArray = []
    counter = 0
    for timeVal in timeArray:
        if (
            timeVal in annotations.onset
        ):  # if the time value is in the onset array, add the corresponding code to the codeArray
            counter += 1
        code_of_target = int(
            codes[counter - 1].replace("T", "")
        )  # convert T0 to 0, T1 to 1, etc
        codeArray.append(code_of_target)

    df["target"] = np.array(codeArray).T
    
    # Convert DataFrame to PyTorch tensors
    signals = torch.tensor(df.drop(columns=['target']).values).float()
    targets = torch.tensor(df['target'].values).long()
    
    return signals, targets


In [14]:
subject = 1
file = 5
fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"

signals, targets = file_to_tensor(fileName)

Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S001/S001R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [16]:
print(signals.shape, targets.shape)

torch.Size([19920, 64]) torch.Size([19920])


In [17]:
def read_all_files_tensor():
    all_signals = []
    all_targets = []
    for subject in range(1, 110):
        for file in range(1, 15):
            fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"
            signals, targets = file_to_tensor(fileName)
            all_signals.append(signals)
            all_targets.append(targets)
    return torch.cat(all_signals), torch.cat(all_targets)

In [134]:
import pickle

def save_tensors_to_pickle(X, y, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump((X, y), f)

def load_tensors_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        X, y = pickle.load(f)
    return X, y

### What is done in experiment

1. Baseline, eyes open
2. Baseline, eyes closed
3. Task 1 (open and close left or right fist)
4. Task 2 (imagine opening and closing left or right fist)
5. Task 3 (open and close both fists or both feet)
6. Task 4 (imagine opening and closing both fists or both feet)
7. Task 1
8. Task 2
9. Task 3
10. Task 4
11. Task 1
12. Task 2
13. Task 3
14. Task 4

In [132]:
def read_all_file_batch(batch_size=640, num_exp=[3, 4], num_people=10):
    all_signals = torch.empty(num_people, len(num_exp), 20000, batch_size, 64)
    all_targets = torch.empty(num_people, len(num_exp), 20000)
    index_exp = 0
    index_people = 0
    for subject in range(1, num_people):
        for file in num_exp:
            fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"
            signals, targets = file_to_tensor(fileName)
            var_signals = torch.empty(20000, batch_size, 64)
            var_targets = torch.empty(20000)
            for i in range(0, 20000):
                if i > batch_size:
                    if i >= len(signals):
                        var_signals[i - batch_size] = torch.full(
                            (batch_size, 64), float("nan")
                        )
                        var_targets[i - batch_size] = float("nan")
                    else:
                        var_signals[i - batch_size] = signals[i - batch_size : i]
                        var_targets[i - batch_size] = targets[i]
            all_signals[index_people][index_exp] = var_signals
            all_targets[index_people][index_exp] = var_targets
            index_exp += 1
        index_people += 1
    return all_signals, all_targets

In [139]:
del signals, targets

In [140]:
all_signals, all_targets = read_all_file_batch(640,[3,7,1],109)
save_tensors_to_pickle(all_signals, all_targets, 'all_signals_targets_batch.pickle')
del all_signals, all_targets

RuntimeError: [enforce fail at alloc_cpu.cpp:117] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 1071513600000 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data_paths):
        self.data_paths = data_paths

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]
        # Załaduj i przetwórz dane z data_path
        data = load_data(data_path)
        return data

dataset = MyDataset(data_paths)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


# AAAAAAAAAAAAAAAAAAAAAAa ja pierdole to się robi dataset z pytorch

In [19]:
all_signals, all_targets = read_all_files_tensor()

Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S001/S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting

/tmp/ipykernel_547721/2070697414.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)
/tmp/ipykernel_547721/2070697414.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)


Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


/tmp/ipykernel_547721/2070697414.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)
/tmp/ipykernel_547721/2070697414.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)


Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


...

Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


/tmp/ipykernel_547721/2070697414.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)
/tmp/ipykernel_547721/2070697414.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)


Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S100/S100R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


/tmp/ipykernel_547721/2070697414.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)
/tmp/ipykernel_547721/2070697414.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  reader = mne.io.read_raw_edf(path, preload=True)


Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S101/S101R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S101/S101R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S101/S101R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/danielkleczykkleczynski/repos/Decoding_of_EEG/files/S101/S101R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting

In [22]:
import pickle

def save_tensors_to_pickle(X, y, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump((X, y), f)

def load_tensors_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        X, y = pickle.load(f)
    return X, y

In [24]:
save_tensors_to_pickle(all_signals, all_targets, 'pkl/all_signals_targets.pkl')
del all_signals, all_targets

In [27]:
all_signals, all_targets = load_tensors_from_pickle('pkl/all_signals_targets.pkl')

In [28]:
all_signals.shape, all_targets.shape

(torch.Size([27756828, 64]), torch.Size([27756828]))